# Machine Learning - Primera parte


## Datos

- *Alumno:* Ricardo Alexis Lopez Cadena
- *Para la materia de:* Clasificación de Datos
- *Universidad de Guadalajara - Centro Universitario de Ciencias Exactas e Ingenierías*
- *Sección D03*

## Objetivo
En la colaboración de un proyecto sobre inventarios de bienes, se desea agregar un módulo de *Machine Learning* para ayudar en base al aprendizaje automático a sugerir clasificadores más pertinentes dentro del catálogo; con tus algoritmos propuestos necesitas encontrar los errores en las clasificaciones y las inconsistencias, para que de esta manera se pueda ajustar el modelo para que sea incorporado a la operación diaria de una aplicación. Ese es el objetivo.

En otras palabras, **encontrar qué items se encuentran en clasificaciones erroneas y corregirlo, además de realizar clasificaciones y subclasificaciones de los items.**

## Problemática
*Es de notar, que lo que se tiene en este archivo son valores únicos por tupla.*

Dentro de este conjunto de datos hay registros que, con toda probabilidad, estén en un lugar equivocado, la intención es buscar los modelos adecuados de algoritmia, para identíficar cuales tuplas con mucha probabilidad están equivocadas.

## Codificación - Proceso de Clasificación

### Estrategía
- **DRY (Don't Repeat Yourself)**
    - Realizar funciones puede ser más complejo debido a que se tiene que pensar en la reusabilidad pero es bastante satisfactorio evitar tareas repetitivas, **el principio DRY permite que el código sea más legible, sea menos código reduciendo así su complejidad y promoviendo la mantenibilidad**

### Módulos requeridos

In [51]:
import pandas as pd # DataFraming
import numpy as np
#import re as regex  # Regular expressions

### Carga de Dataset 
Debido a que el archivo de excel carece de nombres de columnas, se han nombrado las columnas por conveniencia:
- type, primer columna
- class, segunda columna
- description, tercera columna


In [52]:
def loadDataset(fileName):
    return pd.read_excel(fileName)

In [53]:
dataFrame = loadDataset('Actividad.xlsx')
dataFrame.head()

,type,class,description
0,Mobiliario,Mobiliario,SILLON EJECUTIVO RESPALDO MEDIO COLOR NEGRO. N...
1,Mobiliario,Mobiliario,PINTARRON 60X90 (ALTA 20 ENE 1999 MARCA ALFRA).
2,Equipo,Equipo,"MONITOR DE COLOR HP DE 15"", N.D. 017/2002 ALTA..."
3,Mobiliario,Mobiliario,"CESTO DE BASURA ALUMINIO N.D. 007/2005, ALTA 3..."
4,Transporte,Transporte,"AUTOMOVIL FORD ESCORT 1998, PLACAS: HZG1129, N..."


### Contabilidad de tipos

Es necesario comprender cuantos tipos se encuentran en el Dataset.

Nota: en una primera versión de `findTypes()` se encontraba en la lista un item `nan`, por lo que decido deshacerme de este directamente en el ciclo.

In [54]:
def findTypes(typesColumn):
    types = list()

    for type in typesColumn:
        if type not in types and str(type) != 'nan':
            types.append(type)
    
    return types

In [55]:
typesFound = findTypes(dataFrame['type'])

print(f'Los tipos encontrados son {len(typesFound)}: \n')

print('\n'.join(map(str, typesFound)))

Los tipos encontrados son 6: 

Mobiliario
Equipo
Transporte
Bienes CulturalesArtísticos y Científicos
Libro
Propiedad Intelectual


### Limpieza

#### Filas vacías
Limpiando aquellas filas que están vacías o bien les falta al menos un valor.

In [56]:
def cleanDatasetRows(dataFrame):
    def oneValueMissing(dataFrame):
        return dataFrame.dropna()
    def allValuesMissing(dataFrame):
        return dataFrame.dropna(how='all')
    
    return allValuesMissing(oneValueMissing(dataFrame))

In [57]:
"""
Pandas DataFrames interpret empty strings as 'not null' so
    set empty rows to numpy NaN values:
"""
dataFrame["description"].replace('', np.nan, inplace=True)

dataFrame = cleanDatasetRows(dataFrame)
print(f'La limpieza de filas se ha realizado y ahora son: {len(dataFrame)} filas')

La limpieza de filas se ha realizado y ahora son: 9993 filas


#### Filtro de palabras para eliminar el ruido en el nombre del item

In [58]:
# Must be in lowercase:
wordsFilter = set(['de', 'con', 'y', '', 'sin', 'que', 'o', 'a', 'la', 'al',
                   'un', 'una', 'los', 'las', 'por', 'en', 'no.', 'n.d', 'para',
                   's/m', 's/m,', 's/n', 's/n.', 'el', 'del', 'xla',
                   '"a', 'es', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                   '10', '-', '#', 'color', 'muy', 'ni', 'no', 'caracteristicas:',
                   'características:', 'n.d.', 's/n:',''])


#### Columna de descripcion

In [59]:
def cleanSentenceWords(sentence, wordsfilter):
    def cleanWordRecursive(sentence, word):
        if word in sentence:
            sentence.remove(word)
            cleanWordRecursive(sentence, word)
        else:
            return

    sentenceTokens = sentence.casefold().split()
    
    for word in wordsFilter:
        if word in sentenceTokens:
            cleanWordRecursive(sentenceTokens,word)
            
    return ' '.join(sentenceTokens)

# maybe:
#def cleanSentenceByRegEx(sentence, regEx):

In [60]:
def cleanFirstCharacterSpace(word):
    try:
        if word[0] == ' ':
            word = word[1:]
    except Exception as error:
        print(f'a row has escaped empty rows cleansing, leading to: {error}')
    return word

In [61]:
def cleanSentenceCharacters(sentence):
    characters = set(['"', '\'', ',', ', ', ';','#', ':', 'P/'])
    for char in characters:
        sentence = sentence.replace(char,'')
    return sentence

In [62]:
def cleanDescriptionColumn(row):
    row = cleanSentenceWords(row, wordsFilter).upper()
    row = cleanFirstCharacterSpace(row)
    row = cleanSentenceCharacters(row)
    return row 

In [63]:
dataFrameCopy = dataFrame.copy()
dataFrame["description"] = dataFrame["description"].astype(str).apply(cleanDescriptionColumn)

a row has escaped empty rows cleansing, leading to: string index out of range


#### Mostrando diferencias - Antes y después de la limpieza

In [64]:
dataFrameCopy.head()

,type,class,description
0,Mobiliario,Mobiliario,SILLON EJECUTIVO RESPALDO MEDIO COLOR NEGRO. N...
1,Mobiliario,Mobiliario,PINTARRON 60X90 (ALTA 20 ENE 1999 MARCA ALFRA).
2,Equipo,Equipo,"MONITOR DE COLOR HP DE 15"", N.D. 017/2002 ALTA..."
3,Mobiliario,Mobiliario,"CESTO DE BASURA ALUMINIO N.D. 007/2005, ALTA 3..."
4,Transporte,Transporte,"AUTOMOVIL FORD ESCORT 1998, PLACAS: HZG1129, N..."


In [65]:
dataFrame.head()

,type,class,description
0,Mobiliario,Mobiliario,SILLON EJECUTIVO RESPALDO MEDIO NEGRO. ND 13/2...
1,Mobiliario,Mobiliario,PINTARRON 60X90 (ALTA 20 ENE 1999 MARCA ALFRA).
2,Equipo,Equipo,MONITOR HP 15 017/2002 ALTA 07 OCTUBRE 2002
3,Mobiliario,Mobiliario,CESTO BASURA ALUMINIO 007/2005 ALTA 31 AGOSTO ...
4,Transporte,Transporte,AUTOMOVIL FORD ESCORT 1998 PLACAS HZG1129 SERI...


### Clasificación de datos

#### Creando una nueva columna 'item'
Esta servirá para guardar el nombre del item

In [66]:
def getItemName(sentence):
    sentenceTokens = sentence.split()
    return ' '.join(sentenceTokens[:2])

In [67]:
dataFrameCopy = dataFrame.copy()

In [68]:
dataFrameCopy["item"] = dataFrameCopy["description"].astype(str).apply(getItemName)
dataFrameCopy.head()

,type,class,description,item
0,Mobiliario,Mobiliario,SILLON EJECUTIVO RESPALDO MEDIO NEGRO. ND 13/2...,SILLON EJECUTIVO
1,Mobiliario,Mobiliario,PINTARRON 60X90 (ALTA 20 ENE 1999 MARCA ALFRA).,PINTARRON 60X90
2,Equipo,Equipo,MONITOR HP 15 017/2002 ALTA 07 OCTUBRE 2002,MONITOR HP
3,Mobiliario,Mobiliario,CESTO BASURA ALUMINIO 007/2005 ALTA 31 AGOSTO ...,CESTO BASURA
4,Transporte,Transporte,AUTOMOVIL FORD ESCORT 1998 PLACAS HZG1129 SERI...,AUTOMOVIL FORD


#### Exportando a CSV para realizar observaciones
He exportado el proceso realizado hasta ahora para una observación en la columna 'item' y pensar sobre un criterio para clasificación

In [69]:
dataFrameCopy.to_csv('act_processed.csv')

#### Creación de filtros para clasificación

In [70]:
classificationFilters = {
    'Transporte' : {
        'automovil' : ['AUTOMOVIL','PASAJEROS','SEDAN','CAMIONETA','LANCHA','PICKUP','PICK','VEHICULAR''AUTOBUS',
                    'CAMION','MOTO','MOTOCICLETA','VEHICULO','UNIDAD','HONDA','TSURU','CRV','FRONTIER'],
    },
    'Mobiliario' : {
        'oficina' : ['TARJETA','TABLERO','MODULO','NOTEBOOK','NETBOOK','ANALIZADOR','COMPRESOR','ARCHIVERO',
                     'LAMPARA','MAMPARA','ANAQUEL','LIBRERO','PERCHERO','CALENTADOR','SALA','SECRETARIAL',
                    'BASE','SILLA','MUEBLE','FOTOCOPIADORA'],
        'cocina' : ['HORNO','ESTUFA','REFRIGERADOR','MICROONDAS','CAFETERRA','COCINA','ALACENA','CONGELADOR',
                   'PARRILLA'],
        'educacion' : ['PIZZARON','PINTARRON'],
    },
    'Equipo' : {
        'electronico' : ['MONITOR','REGULADOR','TELEFONO','COMPUTADORA','CPU','MOUSE','RATON','TECLADO',
                         'BREAK','NOBREAK','AUDIFONO','MICROFONO','GRABADORA','SCANER','PROYECTOR','LAPTOP',
                        'PANTALLA','LASSER','LASER','RW','MEMORIA','KINGSTON','PENTIUM','USB','PROCESADOR',
                        'CONSOLA','TOSHIBA','SONY','CAMARA','SWITCH','DRIVE','SERVIDOR','RED','TP','ROUTER',
                         'ROUTERS','CONMUTADOR','MODEM','FLOPPY','ELECTRONICO','DIGITAL','TRAKER','TRACKER'
                        'DISCO DURO','IMPRESORA','REGISTRADORA','VIDEO','VIDEOCASETERA'],
        'seguridad' : ['BOTIQUIN','EXTINTOR','EXTINGUIDOR'],
        'mantenimiento' : ['PODADORA','PULIDORA','HERRAMIENTAS'],
        'pintura' : ['PINTURA','ACRILICO','MURAL']
    },
    'Bienes CulturalesArtísticos y Científicos' : {
        'obra artistica' : ['OBRA','LITOGRAFIA'],
        'obra cientifica' : ['DOCUMENTO', 'HISTORICO'],
    },
    'Libro' : {
      'libro' : ['LIBRO','CUENTO','REVISTA','VAQUERO','LUNA DE PLUTON']
    },
    'Propiedad Intelectual' : {
        'licencia' : ['LICENCIA']
    }
}

#### Clasificando y reescribiendo columna 'class'

In [71]:
def matchTypes(parentDictionary, itemName):
    def isMatch(values, itemName):
            for word in itemName:
                if word in values:
                    return True
            return False
        
    for parentType, childDictionary in parentDictionary.items():
        for childType, valuesList in childDictionary.items():
            if isMatch(valuesList, itemName.split()):
                return parentType, childType
    return False, False

In [88]:
## --- NORMAL CLASIFICATION
def classificationOnClass(parentDictionary, row):
    itemType, itemClass = matchTypes(parentDictionary, row["item"])    
    return itemClass if not False else row["class"]
    #if itemClass is not False:
        #return itemClass
    #return row["class"]

## --- CLASIFICATION THAT DISCARDS NOT CLASSIFIED VALUES
def bruteClassificationOnClass(parentDictionary, row):            
    itemType, itemClass = matchTypes(parentDictionary, row["item"])    
    return itemClass if not False else np.nan
    # after this, can call cleanDatasetRows() again

In [89]:
dataFrameCopy['class'] = dataFrameCopy.astype(str).apply(
    lambda row: classificationOnClass(classificationFilters, row), axis=1,
    #lambda row: bruteClassificationOnClass(classificationFilters, row), axis=1
)

#### Limpieza opcional
Si se usó `bruteClassificationOnClass()` se hace una limpieza "Bruta": eliminando filas que no pudieron ser clasificadas

In [90]:
# Before cleansing
#dataFrameCopy.to_csv('dataset_processed.csv')

# After cleansing
#dataFrameCopy = cleanDatasetRows(dataFrameCopy)
#dataFrameCopy.to_csv('dataset_processed_used_nan.csv') 